## Chess AI
construct $f(p)$ as a 3 layer deep 2048 units wide artificial neural network\
for each move, $f(p) = \max\limits_{p\rightarrow p_0} - f(p_0)$\

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import chess
import chess.pgn
import time
import math
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LAMBDA = 0.01
EPOCHS = 10
KAPPA = 10.0

### Prepare dataset
1. Players will choose an optimal or near-optimal move. This means that for two position in succession 
$p \rightarrow q$ observed in the game, we will have $f(p) = -f(q)$
2. For the same reason above, going from $p$ not to $q$, but to a random position $r$, we must have $f(r) > f(q)$ because the random position is better for the next player and worse for the player that made the move.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dimitrioskourtikakis/gm-games-chesscom")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/dimitrioskourtikakis/gm-games-chesscom/versions/1


In [3]:
import pandas
chessGames = pandas.read_csv("dataset.csv", chunksize=100000)
# copy the "pgn" column to a pgn file
f = open("chessGames.pgn", "w")
for chunk in tqdm(chessGames):
    for pgn in chunk['pgn']:
        f.write(pgn + "\n\n")
f.close()

/root/miniconda3/lib/python3.12/site-packages/tqdm/std.py:1181: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
/root/miniconda3/lib/python3.12/site-packages/tqdm/std.py:1181: DtypeWarning: Columns (2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
5it [00:17,  3.59s/it]


KeyboardInterrupt: 

In [ ]:
import random
piece_to_index = {
    'P': 0,  'N': 1,  'B': 2,  'R': 3,  'Q': 4,  'K': 5,  # White pieces
    'p': 6,  'n': 7,  'b': 8,  'r': 9,  'q': 10, 'k': 11,  # Black pieces
}
class ChessDataset(Dataset):
    def __init__(self, games, device):
        self.games = games
        self.device = device

    def __len__(self):
        return len(self.games)
        
    def board2vec(board):
        vec = torch.zeros((12, 8, 8), dtype=torch.float32).to(device)  # 调整维度顺序
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None:
                vec[piece_to_index[piece.symbol()]][square // 8][square % 8] = 1
        return vec

    def __getitem__(self, idx):
        # Find the game and move index corresponding to idx
        current_idx = 0
        for game in self.games:
            moves = list(game.mainline_moves())
            if current_idx + len(moves) > idx:
                move_idx = idx - current_idx
                chessBoard = game.board()

                # Play up to the move_idx
                for move in moves[:move_idx]:
                    chessBoard.push(move)

                # Generate p, q, and r
                p = self.board2vec(chessBoard)
                legal_moves = list(chessBoard.legal_moves)
                pseudo_move = random.choice(legal_moves)
                chessBoard2 = chessBoard.copy()
                chessBoard2.push(pseudo_move)
                r = self.board2vec(chessBoard2)
                chessBoard.push(moves[move_idx])
                q = self.board2vec(chessBoard)
                return p, q, r
            current_idx += len(moves)
        raise IndexError("Index out of range")
        
with open("chessGames.pgn") as f:
    games = list()
    while game:=chess.pgn.read_game(f):
        games.append(game)

dataloader = ChessDataset(games, device)

In [ ]:
class ChessValueNetwork(nn.Module):
    def __init__(self):
        super(ChessValueNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(12, 32, kernel_size=3, padding=1), # [12, 8, 8] -> [32, 8, 8]
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # [32, 8, 8] -> [64, 8, 8]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2) # [64, 8, 8] -> [64, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(1024, 256), # [1, 64*4*4] -> [1, 256]
            nn.ReLU(),
            nn.Linear(256, 128), # [1, 256] -> [1, 128]
            nn.ReLU(),
            nn.Linear(128, 1),  # 输出标量
            nn.Tanh()  # 限制在 [-1, 1]
        )

    def forward(self, x):
        x = self.conv(x)
        # 展平成 [Features]
        x = x.view(-1)
        x = self.fc(x)
        return x
    
# 目标函数定义
import torch
import torch.nn.functional as F

def objective_function(model, p, q, r, kappa=10.0):
    # Forward pass: Compute scores for p, q, and r
    f_p = model(p).squeeze()  # Score for p
    f_q = model(q).squeeze()  # Score for q
    f_r = model(r).squeeze()  # Score for r
    
    # Loss components
    # Loss A: Ensure f(q) > f(r) (optimal move vs random move)
    loss_a = -torch.log(F.sigmoid(f_q - f_r)).mean()

    # Loss B: Ensure f(p) + f(q) close to zero (soft equality constraint)
    loss_b = -torch.log(F.sigmoid(kappa * (f_p + f_q))).mean()

    # Loss C: Ensure -f(p) - f(q) close to zero (soft equality constraint)
    loss_c = -torch.log(F.sigmoid(-kappa * (f_p + f_q))).mean()

    # Total loss: Combine all components
    total_loss = loss_a + loss_b + loss_c

    return total_loss

def lr_lambda(current_epoch):
    current_time = time.time()  # 当前时间戳
    elapsed_time = current_time - t0
    return math.exp(-elapsed_time / 86400)

model = ChessValueNetwork().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=LAMBDA)
t0 =time.time()
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


In [ ]:
for epoch in range(EPOCHS):
    total_loss = 0
    for p, q, r in dataloader:
        optimizer.zero_grad()
        loss = objective_function(model, p, q, r, KAPPA).to(device)
        loss.backward()
        print(loss)
        optimizer.step()
        scheduler.step()   
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")